In [ ]:
!pip install gym
# Example installation
!pip install ortools
!pip install -U -q PyDrive
!pip install imageio
# Example installation
!pip install deap
# Example installation
!pip install acopy
!pip install pyswarm
!pip install pyswarms
!pip install numpy pandas matplotlib plotly networkx
!pip install platypus-opt
#!pip install --upgrade ipython jupyter




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.17.0 requires protobuf!=4.24.0,<5.0.0,>=3.19.6, but you have protobuf 5.26.1 which is incompatible.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.26.1 which is incom

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
from itertools import product

# Function to calculate Euclidean distance between two points
def euclidean_distance(p1, p2):
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# Parse the dataset from c101.txt file
def parse_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file.readlines()[9:]:  # Skip header lines
            fields = line.strip().split()
            if len(fields) < 7:
                continue
            customer = {
                "customer_id": int(fields[0]),
                "x": float(fields[1]),
                "y": float(fields[2]),
                "demand": int(fields[3]),
                "ready_time": int(fields[4]),
                "due_time": int(fields[5]),
                "service_time": int(fields[6])
            }
            data.append(customer)
    return pd.DataFrame(data)

class ACO:
    def __init__(self, locations, vehicle_capacity, num_ants=25, alpha=1, beta=1, evaporation_rate=0.5):
        self.locations = locations  # locations should be a list of tuples (x, y)
        self.vehicle_capacity = vehicle_capacity
        self.num_ants = num_ants
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.num_customers = len(locations)
        self.pheromone = np.ones((self.num_customers, self.num_customers))
        self.best_route = None
        self.best_distance = float('inf')

    def construct_route(self):
        route = [0]  # Start at the depot (customer 0)
        visited = [False] * self.num_customers
        visited[0] = True

        while len(route) < self.num_customers:
            current = route[-1]
            next_customer = self.select_next_customer(current, visited)
            route.append(next_customer)
            visited[next_customer] = True

        route_distance = self.calculate_distance(route)

        return route, route_distance

    def select_next_customer(self, current, visited):
        probabilities = []
        total_pheromone = 0
        # Loop over all customers
        for j in range(self.num_customers):
            if not visited[j]:
                dist = euclidean_distance(self.locations[current], self.locations[j])
                pheromone = self.pheromone[current, j] ** self.alpha  # Influence of pheromone
                prob = pheromone / (dist ** self.beta) if dist != 0 else 0
                total_pheromone += prob
                probabilities.append(prob)

        # Normalize the probabilities so that they sum to 1
        probabilities = [p / total_pheromone for p in probabilities] if total_pheromone > 0 else probabilities

        # Select the next customer based on the probabilities
        unvisited_customers = [j for j in range(self.num_customers) if not visited[j]]
        return np.random.choice(unvisited_customers, p=probabilities)

    def calculate_distance(self, route):
        total_distance = 0
        for i in range(len(route) - 1):
            from_customer = self.locations[route[i]]
            to_customer = self.locations[route[i + 1]]
            total_distance += euclidean_distance(from_customer, to_customer)
        return total_distance

    def update_pheromone(self, route, distance):
        # Update pheromone levels (evaporation + deposit)
        self.pheromone = (1 - self.evaporation_rate) * self.pheromone
        for i in range(len(route) - 1):
            self.pheromone[route[i], route[i + 1]] += 1 / distance

    def run(self, iterations):
        for _ in range(iterations):
            all_routes = []
            for _ in range(self.num_ants):
                route, route_distance = self.construct_route()
                all_routes.append((route, route_distance))
                if route_distance < self.best_distance:
                    self.best_route = route
                    self.best_distance = route_distance
            self.update_pheromone(self.best_route, self.best_distance)
        return self.best_route, self.best_distance

# PSO Class (Particle Swarm Optimization)
import numpy as np

class PSO:
    def __init__(self, locations, vehicle_capacity, num_particles=5, inertia=0.5, cognitive=1.5, social=1.5):
        self.locations = locations  # locations should be a list of tuples (x, y)
        self.vehicle_capacity = vehicle_capacity
        self.num_particles = num_particles
        self.global_best_route = None
        self.global_best_distance = float('inf')
        self.inertia = inertia
        self.cognitive = cognitive
        self.social = social

    def run(self, iterations):
        # Initialize particles as random permutations of customers
        particles = [np.random.permutation(len(self.locations)) for _ in range(self.num_particles)]
        velocities = [np.zeros(len(self.locations)) for _ in range(self.num_particles)]  # Initialize velocities

        for _ in range(iterations):
            for i, particle in enumerate(particles):
                # Calculate distance for current particle (route)
                candidate_distance = self.calculate_distance(particle)

                # If the new route is better, update the global best
                if candidate_distance < self.global_best_distance:
                    self.global_best_route = particle
                    self.global_best_distance = candidate_distance

                # Update particle velocity and position (swap mutation strategy)
                velocities[i] = self.update_velocity(particles[i], velocities[i], i)
                particles[i] = self.update_position(particles[i], velocities[i])

        return self.global_best_route, self.global_best_distance

    def calculate_distance(self, route):
        # Calculate the total travel distance of the route
        total_distance = 0
        for i in range(len(route) - 1):
            from_customer = self.locations[route[i]]  # Accessing customer coordinates
            to_customer = self.locations[route[i + 1]]
            total_distance += euclidean_distance(from_customer, to_customer)
        return total_distance

    def update_velocity(self, particle, velocity, particle_index):
        # Use a simple heuristic to determine the "change" in position based on cognitive and social factors
        r1, r2 = np.random.random(2)
        velocity_change = np.zeros(len(self.locations))

        # Simple swap-based mutation in velocity
        for i in range(len(particle) - 1):
            if np.random.rand() < self.inertia:
                # Select two positions to swap based on random factors
                swap_idx1, swap_idx2 = np.random.choice(len(particle), 2, replace=False)
                velocity_change[swap_idx1] = 1
                velocity_change[swap_idx2] = -1

        return velocity_change

    def update_position(self, particle, velocity):
        # Apply swap mutation to the particle's route
        for i in range(len(velocity)):
            if velocity[i] == 1:
                swap_idx = np.where(velocity == -1)[0][0]
                particle[i], particle[swap_idx] = particle[swap_idx], particle[i]
        return particle

# Helper function to calculate Euclidean distance
def euclidean_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)


# Hybrid Class combining ACO and PSO
class HybridACO_PSO:
    def __init__(self, aco, pso):
        self.aco = aco
        self.pso = pso

    def optimize(self, aco_iterations, pso_iterations):
        # Run ACO optimization
        best_route_aco, best_distance_aco = self.aco.run(aco_iterations)
        # Run PSO optimization
        best_route_pso, best_distance_pso = self.pso.run(pso_iterations)

        # Combine results or use hybrid approach (this is a placeholder logic)
        hybrid_best_route = best_route_aco if best_distance_aco < best_distance_pso else best_route_pso
        hybrid_best_distance = min(best_distance_aco, best_distance_pso)

        return hybrid_best_route, hybrid_best_distance

# Function to count time window violations
def count_time_window_violations(route, time_windows, service_times):
    violations = 0
    current_time = 0
    for i in range(len(route)):
        customer = route[i]
        ready_time, due_time = time_windows[customer]
        service_time = service_times[customer]

        if current_time < ready_time:
            current_time = ready_time

        current_time += service_time

        if current_time > due_time:
            violations += 1
    return violations

import pandas as pd

def parse_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file.readlines()[9:]:  # Assuming the data starts after a header
            fields = line.strip().split()
            if len(fields) < 7:
                continue
            customer = {
                "customer_id": int(fields[0]),
                "x": float(fields[1]),
                "y": float(fields[2]),
                "demand": int(fields[3]),
                "ready_time": int(fields[4]),
                "due_time": int(fields[5]),
                "service_time": int(fields[6])
            }
            data.append(customer)
    return pd.DataFrame(data)

# Example usage
dataset = parse_dataset("/content/drive/MyDrive/c101.txt")

# Main code
if __name__ == "__main__":
    # Load data from the file (c101.txt)
    file_path = '/content/drive/MyDrive/c101.txt'  # Adjust the path accordingly
    dataset = parse_dataset(file_path)

    # Extract depot and customer data
    depot = (dataset.iloc[0]['x'], dataset.iloc[0]['y'])  # Assume depot is the first row
    locations = dataset[['x', 'y']].values  # Get all locations including depot if needed
    demands = dataset['demand'].values      # Get demands for all entries
    time_windows = dataset[['ready_time', 'due_time']].values  # Time windows
    service_times = dataset['service_time'].values  # Service times
    vehicle_capacity = 200  # Example vehicle capacity (adjust as needed)
    num_customers = len(locations)

    # ACO and PSO parameter configurations
    ACO_CONFIGS = {
        'num_ants': [25],
        'alpha': [1, 2],
        'beta': [2, 3, 5],
        'evaporation_rate': [0.4, 0.6, 0.8]
    }

    PSO_CONFIGS = {
        'num_particles': [15, 25, 35],
        'inertia': [0.5, 0.7, 0.9],
        'cognitive': [1.5, 2.5],
        'social': [1.5, 2.5, 3.0]
    }

    # Initialize lists to store results
    results_aco = []
    results_pso = []
    results_hybrid = []

    # Generate all combinations of ACO and PSO parameters
    aco_combinations = list(product(*ACO_CONFIGS.values()))
    pso_combinations = list(product(*PSO_CONFIGS.values()))

    # Iterate over each combination of ACO and PSO parameters
    for aco_config in aco_combinations:
        for pso_config in pso_combinations:
            aco_params = {
                'num_ants': aco_config[0],
                'alpha': aco_config[1],
                'beta': aco_config[2],
                'evaporation_rate': aco_config[3]
            }
            pso_params = {
                'num_particles': pso_config[0],
                'inertia': pso_config[1],
                'cognitive': pso_config[2],
                'social': pso_config[3]
            }

            # Run ACO
            aco = ACO(locations, vehicle_capacity, **aco_params)
            best_route_aco, best_distance_aco = aco.run(50)
            violations_aco = count_time_window_violations(best_route_aco, time_windows, service_times)
            results_aco.append({
                'config': aco_config + pso_config,
                'best_distance': best_distance_aco,
                'violations': violations_aco
            })

            # Run PSO
            pso = PSO(locations, vehicle_capacity, **pso_params)
            best_route_pso, best_distance_pso = pso.run(50)
            violations_pso = count_time_window_violations(best_route_pso, time_windows, service_times)
            results_pso.append({
                'config': aco_config + pso_config,
                'best_distance': best_distance_pso,
                'violations': violations_pso
            })

            # Initialize HybridACO_PSO with separate ACO and PSO instances
            hybrid = HybridACO_PSO(aco, pso)
            best_route_hybrid, best_distance_hybrid = hybrid.optimize(50, 50)
            violations_hybrid = count_time_window_violations(best_route_hybrid, time_windows, service_times)
            results_hybrid.append({
                'config': aco_config + pso_config,
                'best_distance': best_distance_hybrid,
                'violations': violations_hybrid
            })

    # Convert results to DataFrames
    df_aco = pd.DataFrame(results_aco)
    df_pso = pd.DataFrame(results_pso)
    df_hybrid = pd.DataFrame(results_hybrid)

    # Save results to CSV (optional)
    df_aco.to_csv('results_aco.csv', index=False)
    df_pso.to_csv('results_pso.csv', index=False)
    df_hybrid.to_csv('results_hybrid.csv', index=False)

    # Plot results
    plot_best_route(locations, best_route_aco)
    plot_best_route(locations, best_route_pso)
    plot_best_route(locations, hybrid_best_route)
    ax[0].set_title('ACO Best Route')
    ax[1].set_title('PSO Best Route')
    ax[2].set_title('Hybrid ACO-PSO Best Route')

    fig, ax = plt.subplots(1, 3, figsize=(18, 5))


    ax[0].scatter(df_aco['best_distance'], df_aco['violations'], color='blue', label='ACO')
    ax[0].set_title('ACO Results')
    ax[0].set_xlabel('Best Distance')
    ax[0].set_ylabel('Time Window Violations')

    ax[1].scatter(df_pso['best_distance'], df_pso['violations'], color='green', label='PSO')
    ax[1].set_title('PSO Results')
    ax[1].set_xlabel('Best Distance')
    ax[1].set_ylabel('Time Window Violations')

    ax[2].scatter(df_hybrid['best_distance'], df_hybrid['violations'], color='red', label='Hybrid ACO-PSO')
    ax[2].set_title('Hybrid ACO-PSO Results')
    ax[2].set_xlabel('Best Distance')
    ax[2].set_ylabel('Time Window Violations')

    plt.tight_layout()
    plt.show()


In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
from itertools import product

# Function to calculate Euclidean distance between two points
def euclidean_distance(p1, p2):
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)


# Example configuration dictionary
config = {
    "ACO": {
        "num_vehicles": 25,
        "num_ants": 10,
        "alpha": 1.0,
        "beta": 1.0,
        "evaporation_rate": 0.5,
        "iterations": 100
    },
    "PSO": {
        "num_particles": 25,
        "inertia": 0.5,
        "cognitive": 1.5,
        "social": 1.5,
        "iterations": 100
    },
    "HybridACO_PSO": {
        "aco_iterations": 100,
        "pso_iterations": 100
    }
}

# Parse the dataset from c101.txt file
def parse_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file.readlines()[9:]:  # Skip header lines
            fields = line.strip().split()
            if len(fields) < 7:
                continue
            customer = {
                "customer_id": int(fields[0]),
                "x": float(fields[1]),
                "y": float(fields[2]),
                "demand": int(fields[3]),
                "ready_time": int(fields[4]),
                "due_time": int(fields[5]),
                "service_time": int(fields[6])
            }
            data.append(customer)
    return pd.DataFrame(data)


class ACO:
    def __init__(self, locations, vehicle_capacity, config):
        self.locations = locations
        self.vehicle_capacity = vehicle_capacity
        self.num_vehicles = config['num_vehicles']
        self.num_ants = config['num_ants']
        self.alpha = config['alpha']
        self.beta = config['beta']
        self.evaporation_rate = config['evaporation_rate']
        self.num_customers = len(locations)
        self.pheromone = np.ones((self.num_customers, self.num_customers))
        self.best_route = None
        self.best_distance = float('inf')

    def construct_route(self):
        route = [0]  # Start at the depot (customer 0)
        visited = [False] * self.num_customers
        visited[0] = True

        while len(route) < self.num_customers:
            current = route[-1]
            next_customer = self.select_next_customer(current, visited)
            route.append(next_customer)
            visited[next_customer] = True

        route_distance = self.calculate_distance(route)
        return route, route_distance

    def select_next_customer(self, current, visited):
        probabilities = []
        total_pheromone = 0
        for j in range(self.num_customers):
            if not visited[j]:
                dist = euclidean_distance(self.locations[current], self.locations[j])
                pheromone = self.pheromone[current, j] ** self.alpha
                prob = pheromone / (dist ** self.beta) if dist != 0 else 0
                total_pheromone += prob
                probabilities.append(prob)

        probabilities = [p / total_pheromone for p in probabilities] if total_pheromone > 0 else probabilities
        unvisited_customers = [j for j in range(self.num_customers) if not visited[j]]
        return np.random.choice(unvisited_customers, p=probabilities)

    def calculate_distance(self, route):
        total_distance = 0
        for i in range(len(route) - 1):
            from_customer = self.locations[route[i]]
            to_customer = self.locations[route[i + 1]]
            total_distance += euclidean_distance(from_customer, to_customer)
        return total_distance

    def update_pheromone(self, route, distance):
        self.pheromone = (1 - self.evaporation_rate) * self.pheromone
        for i in range(len(route) - 1):
            self.pheromone[route[i], route[i + 1]] += 1 / distance

    def run(self):
        for _ in range(self.num_ants):
            all_routes = []
            for _ in range(self.num_ants):
                route, route_distance = self.construct_route()
                all_routes.append((route, route_distance))
                if route_distance < self.best_distance:
                    self.best_route = route
                    self.best_distance = route_distance
            self.update_pheromone(self.best_route, self.best_distance)
        return self.best_route, self.best_distance


class PSO:
    def __init__(self, locations, vehicle_capacity, config):
        self.locations = locations
        self.vehicle_capacity = vehicle_capacity
        self.num_particles = config['num_particles']
        self.inertia = config['inertia']
        self.cognitive = config['cognitive']
        self.social = config['social']
        self.global_best_route = None
        self.global_best_distance = float('inf')

    def run(self):
        particles = [np.random.permutation(len(self.locations)) for _ in range(self.num_particles)]
        velocities = [np.zeros(len(self.locations)) for _ in range(self.num_particles)]

        for _ in range(config["PSO"]["iterations"]):
            for i, particle in enumerate(particles):
                candidate_distance = self.calculate_distance(particle)
                if candidate_distance < self.global_best_distance:
                    self.global_best_route = particle
                    self.global_best_distance = candidate_distance

                velocities[i] = self.update_velocity(particles[i], velocities[i], i)
                particles[i] = self.update_position(particles[i], velocities[i])

        return self.global_best_route, self.global_best_distance

    def calculate_distance(self, route):
        total_distance = 0
        for i in range(len(route) - 1):
            from_customer = self.locations[route[i]]
            to_customer = self.locations[route[i + 1]]
            total_distance += euclidean_distance(from_customer, to_customer)
        return total_distance

    def update_velocity(self, particle, velocity, particle_index):
        r1, r2 = np.random.random(2)
        velocity_change = np.zeros(len(self.locations))

        for i in range(len(particle) - 1):
            if np.random.rand() < self.inertia:
                swap_idx1, swap_idx2 = np.random.choice(len(particle), 2, replace=False)
                velocity_change[swap_idx1] = 1
                velocity_change[swap_idx2] = -1

        return velocity_change

    def update_position(self, particle, velocity):
        for i in range(len(velocity)):
            if velocity[i] == 1:
                swap_idx = np.where(velocity == -1)[0][0]
                particle[i], particle[swap_idx] = particle[swap_idx], particle[i]
        return particle


class HybridACO_PSO:
    def __init__(self, aco, pso, config):
        self.aco = aco
        self.pso = pso
        self.config = config

    def optimize(self):
        aco_best_route, aco_best_distance = self.aco.run()
        pso_best_route, pso_best_distance = self.pso.run()

        hybrid_best_route = aco_best_route if aco_best_distance < pso_best_distance else pso_best_route
        hybrid_best_distance = min(aco_best_distance, pso_best_distance)

        return hybrid_best_route, hybrid_best_distance


# Main code to execute everything
if __name__ == "__main__":
    # Load data
    file_path = '/content/drive/MyDrive/c101.txt'
    df = parse_dataset(file_path)

    # Define locations, demands, and time windows from the dataframe
    locations = [(row['x'], row['y']) for _, row in df.iterrows()]
    time_windows = [(row['ready_time'], row['due_time']) for _, row in df.iterrows()]
    service_times = list(df['service_time'])

    # Initialize ACO and PSO optimization classes using configurations
    aco = ACO(locations, vehicle_capacity=200, config=config['ACO'])
    pso = PSO(locations, vehicle_capacity=200, config=config['PSO'])

    hybrid = HybridACO_PSO(aco, pso, config['HybridACO_PSO'])

    # Run the algorithms with the appropriate iterations
    aco_best_route, aco_best_distance = aco.run()
    pso_best_route, pso_best_distance = pso.run()

    hybrid_best_route, hybrid_best_distance = hybrid.optimize()

    print(f'ACO Best Distance: {aco_best_distance}')
    print(f'PSO Best Distance: {pso_best_distance}')
    print(f'Hybrid Best Distance: {hybrid_best_distance}')


ACO Best Distance: 1953.7729181086177
PSO Best Distance: 3268.095916914318
Hybrid Best Distance: 986.8259499977064


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt
from itertools import product

# Function to calculate Euclidean distance between two points
def euclidean_distance(p1, p2):
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# Parse the dataset from c101.txt file
def parse_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file.readlines()[9:]:  # Skip header lines
            fields = line.strip().split()
            if len(fields) < 7:
                continue
            customer = {
                "customer_id": int(fields[0]),
                "x": float(fields[1]),
                "y": float(fields[2]),
                "demand": int(fields[3]),
                "ready_time": int(fields[4]),
                "due_time": int(fields[5]),
                "service_time": int(fields[6])
            }
            data.append(customer)
    return pd.DataFrame(data)
import numpy as np
from math import sqrt
import random

# Function to calculate Euclidean distance between two points
def euclidean_distance(p1, p2):
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# ACO (Ant Colony Optimization) for Vehicle Routing Problem
class ACO:
    def __init__(self, locations, vehicle_capacity, num_vehicles, num_ants=10, alpha=1, beta=1, evaporation_rate=0.5):
        self.locations = locations  # Locations should be a list of tuples (x, y)
        self.vehicle_capacity = vehicle_capacity
        self.num_vehicles = num_vehicles
        self.num_ants = num_ants
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.num_customers = len(locations)
        self.pheromone = np.ones((self.num_customers, self.num_customers))
        self.best_route = None
        self.best_distance = float('inf')

    def construct_route(self):
        route = [[] for _ in range(self.num_vehicles)]  # List of routes for each vehicle
        visited = [False] * self.num_customers
        visited[0] = True  # Start at depot

        for vehicle in range(self.num_vehicles):
            current = route[vehicle][-1] if route[vehicle] else 0
            while len([i for i in range(1, self.num_customers) if not visited[i]]) > 0:
                next_customer = self.select_next_customer(current, visited)
                if next_customer is not None:
                    route[vehicle].append(next_customer)
                    visited[next_customer] = True
        return route

    def select_next_customer(self, current, visited):
        probabilities = []
        total_pheromone = 0
        for j in range(self.num_customers):
            if not visited[j]:
                dist = euclidean_distance(self.locations[current], self.locations[j])
                pheromone = self.pheromone[current, j] ** self.alpha
                prob = pheromone / (dist ** self.beta) if dist != 0 else 0
                total_pheromone += prob
                probabilities.append(prob)

        if total_pheromone == 0:
            return None  # No valid customers can be visited

        probabilities = [p / total_pheromone for p in probabilities]
        return np.random.choice([i for i in range(self.num_customers) if not visited[i]], p=probabilities)

    def update_pheromone(self, route, distance):
        # Update pheromone levels (evaporation + deposit)
        self.pheromone = (1 - self.evaporation_rate) * self.pheromone
        for r in route:
            for i in range(len(r) - 1):
                self.pheromone[r[i], r[i + 1]] += 1 / distance

    def run(self, iterations):
        for _ in range(iterations):
            all_routes = []
            for _ in range(self.num_ants):
                route = self.construct_route()
                route_distance = self.calculate_route_distance(route)
                all_routes.append((route, route_distance))
                if route_distance < self.best_distance:
                    self.best_route = route
                    self.best_distance = route_distance
            self.update_pheromone(self.best_route, self.best_distance)
        return self.best_route, self.best_distance

    def calculate_route_distance(self, route):
        total_distance = 0
        for r in route:
            for i in range(len(r) - 1):
                from_customer = self.locations[r[i]]
                to_customer = self.locations[r[i + 1]]
                total_distance += euclidean_distance(from_customer, to_customer)
        return total_distance


# PSO (Particle Swarm Optimization) for Vehicle Routing Problem
class PSO:
    def __init__(self, locations, vehicle_capacity, num_vehicles, num_particles=30, inertia_weight=0.5, cognitive_weight=1.5, social_weight=1.5):
        self.locations = locations
        self.vehicle_capacity = vehicle_capacity
        self.num_vehicles = num_vehicles
        self.num_particles = num_particles
        self.inertia_weight = inertia_weight
        self.cognitive_weight = cognitive_weight
        self.social_weight = social_weight
        self.num_customers = len(locations)
        self.best_route = None
        self.best_distance = float('inf')
        self.particles = []
        self.velocities = []
        self.best_particle_positions = []
        self.best_particle_values = []

        for _ in range(self.num_particles):
            position = self.random_solution()
            self.particles.append(position)
            value = self.calculate_route_distance(position)
            self.best_particle_positions.append(position)
            self.best_particle_values.append(value)
            if value < self.best_distance:
                self.best_route = position
                self.best_distance = value
            self.velocities.append(np.zeros_like(position))

    def random_solution(self):
        route = [[] for _ in range(self.num_vehicles)]
        customers = list(range(1, self.num_customers))
        np.random.shuffle(customers)
        for customer in customers:
            assigned_vehicle = np.random.randint(0, self.num_vehicles)
            route[assigned_vehicle].append(customer)
        return route

    def calculate_route_distance(self, route):
        total_distance = 0
        for vehicle_route in route:
            for i in range(len(vehicle_route) - 1):
                from_customer = self.locations[vehicle_route[i]]
                to_customer = self.locations[vehicle_route[i + 1]]
                total_distance += euclidean_distance(from_customer, to_customer)
        return total_distance

    def update_velocity(self, particle_idx):
        for i in range(self.num_vehicles):
            for j in range(len(self.particles[particle_idx][i]) - 1):
                self.velocities[particle_idx][i][j] = (
                    self.inertia_weight * self.velocities[particle_idx][i][j]
                    + self.cognitive_weight * np.random.random() * (self.best_particle_positions[particle_idx][i][j] - self.particles[particle_idx][i][j])
                    + self.social_weight * np.random.random() * (self.best_route[i][j] - self.particles[particle_idx][i][j])
                )

    def update_position(self, particle_idx):
        for i in range(self.num_vehicles):
            for j in range(len(self.particles[particle_idx][i]) - 1):
                self.particles[particle_idx][i][j] += self.velocities[particle_idx][i][j]
                if sum(self.particles[particle_idx][i]) > self.vehicle_capacity:
                    self.particles[particle_idx][i][j] = self.best_particle_positions[particle_idx][i][j]

    def optimize(self, iterations):
        for _ in range(iterations):
            for i in range(self.num_particles):
                self.update_velocity(i)
                self.update_position(i)
                new_value = self.calculate_route_distance(self.particles[i])

                if new_value < self.best_particle_values[i]:
                    self.best_particle_values[i] = new_value
                    self.best_particle_positions[i] = self.particles[i]

                if new_value < self.best_distance:
                    self.best_distance = new_value
                    self.best_route = self.particles[i]
        return self.best_route, self.best_distance


# ACO-PSO Hybrid Algorithm for Vehicle Routing Problem
class ACO_PSO_Hybrid:
    def __init__(self, locations, vehicle_capacity, num_vehicles, num_ants=10, num_particles=30, alpha=1, beta=1, evaporation_rate=0.5, max_iterations=100, inertia_weight=0.5, cognitive_weight=1.5, social_weight=1.5):
        self.locations = locations
        self.vehicle_capacity = vehicle_capacity
        self.num_vehicles = num_vehicles
        self.num_ants = num_ants
        self.num_particles = num_particles
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.max_iterations = max_iterations
        self.inertia_weight = inertia_weight
        self.cognitive_weight = cognitive_weight
        self.social_weight = social_weight

        self.aco = ACO(locations, vehicle_capacity, num_vehicles, num_ants, alpha, beta, evaporation_rate)
        self.pso = PSO(locations, vehicle_capacity, num_vehicles, num_particles, inertia_weight, cognitive_weight, social_weight)

    def run(self):
        for _ in range(self.max_iterations):
            # Run ACO optimization
            aco_route, aco_distance = self.aco.run(self.max_iterations)
            # Update PSO optimization with the solution from ACO
            self.pso.best_route, self.pso.best_distance = self.pso.optimize(self.max_iterations)
            # Combine ACO and PSO results
            if self.pso.best_distance < self.aco.best_distance:
                self.aco.best_route = self.pso.best_route
                self.aco.best_distance = self.pso.best_distance
        return self.aco.best_route, self.aco.best_distance

# Function to count time window violations
def count_time_window_violations(route, time_windows, service_times):
    violations = 0
    current_time = 0
    for i in range(len(route)):
        customer = route[i]
        ready_time, due_time = time_windows[customer]
        service_time = service_times[customer]

        if current_time < ready_time:
            current_time = ready_time

        current_time += service_time

        if current_time > due_time:
            violations += 1
    return violations

import pandas as pd

def parse_dataset(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file.readlines()[9:]:  # Assuming the data starts after a header
            fields = line.strip().split()
            if len(fields) < 7:
                continue
            customer = {
                "customer_id": int(fields[0]),
                "x": float(fields[1]),
                "y": float(fields[2]),
                "demand": int(fields[3]),
                "ready_time": int(fields[4]),
                "due_time": int(fields[5]),
                "service_time": int(fields[6])
            }
            data.append(customer)
    return pd.DataFrame(data)

# Example usage
dataset = parse_dataset("c101.txt")

# Main code
if __name__ == "__main__":
    # Load data from the file (c101.txt)
    file_path = 'c101.txt'  # Adjust the path accordingly
    dataset = parse_dataset(file_path)

    # Extract depot and customer data
    depot = (dataset.iloc[0]['x'], dataset.iloc[0]['y'])  # Assume depot is the first row
    locations = dataset[['x', 'y']].values  # Get all locations including depot if needed
    demands = dataset['demand'].values      # Get demands for all entries
    time_windows = dataset[['ready_time', 'due_time']].values  # Time windows
    service_times = dataset['service_time'].values  # Service times
    vehicle_capacity = 200  # Example vehicle capacity (adjust as needed)
    num_customers = len(locations)

    # ACO and PSO parameter configurations
    ACO_CONFIGS = {
        'num_ants': [10, 20, 30],
        'alpha': [1, 2],
        'beta': [2, 3, 5],
        'evaporation_rate': [0.4, 0.6, 0.8]
    }

    PSO_CONFIGS = {
        'num_particles': [15, 25, 35],
        'inertia': [0.5, 0.7, 0.9],
        'cognitive': [1.5, 2.5],
        'social': [1.5, 2.5, 3.0]
    }

    # Initialize lists to store results
    results_aco = []
    results_pso = []
    results_hybrid = []

    # Generate all combinations of ACO and PSO parameters
    aco_combinations = list(product(*ACO_CONFIGS.values()))
    pso_combinations = list(product(*PSO_CONFIGS.values()))

    # Iterate over each combination of ACO and PSO parameters
    for aco_config in aco_combinations:
        for pso_config in pso_combinations:
            aco_params = {
                'num_ants': aco_config[0],
                'alpha': aco_config[1],
                'beta': aco_config[2],
                'evaporation_rate': aco_config[3]
            }
            pso_params = {
                'num_particles': pso_config[0],
                'inertia': pso_config[1],
                'cognitive': pso_config[2],
                'social': pso_config[3]
            }

            # Run ACO
            aco = ACO(locations, vehicle_capacity, **aco_params)
            best_route_aco, best_distance_aco = aco.run(50)
            violations_aco = count_time_window_violations(best_route_aco, time_windows, service_times)
            results_aco.append({
                'config': aco_config + pso_config,
                'best_distance': best_distance_aco,
                'violations': violations_aco
            })

            # Run PSO
            pso = PSO(locations, vehicle_capacity, **pso_params)
            best_route_pso, best_distance_pso = pso.run(50)
            violations_pso = count_time_window_violations(best_route_pso, time_windows, service_times)
            results_pso.append({
                'config': aco_config + pso_config,
                'best_distance': best_distance_pso,
                'violations': violations_pso
            })

            # Initialize HybridACO_PSO with separate ACO and PSO instances
            hybrid = HybridACO_PSO(aco, pso)
            best_route_hybrid, best_distance_hybrid = hybrid.optimize(50, 50)
            violations_hybrid = count_time_window_violations(best_route_hybrid, time_windows, service_times)
            results_hybrid.append({
                'config': aco_config + pso_config,
                'best_distance': best_distance_hybrid,
                'violations': violations_hybrid
            })

    # Convert results to DataFrames
    df_aco = pd.DataFrame(results_aco)
    df_pso = pd.DataFrame(results_pso)
    df_hybrid = pd.DataFrame(results_hybrid)

    # Save results to CSV (optional)
    df_aco.to_csv('results_aco.csv', index=False)
    df_pso.to_csv('results_pso.csv', index=False)
    df_hybrid.to_csv('results_hybrid.csv', index=False)

    # Plot results
    fig, ax = plt.subplots(1, 3, figsize=(18, 5))

    ax[0].scatter(df_aco['best_distance'], df_aco['violations'], color='blue', label='ACO')
    ax[0].set_title('ACO Results')
    ax[0].set_xlabel('Best Distance')
    ax[0].set_ylabel('Time Window Violations')

    ax[1].scatter(df_pso['best_distance'], df_pso['violations'], color='green', label='PSO')
    ax[1].set_title('PSO Results')
    ax[1].set_xlabel('Best Distance')
    ax[1].set_ylabel('Time Window Violations')

    ax[2].scatter(df_hybrid['best_distance'], df_hybrid['violations'], color='red', label='Hybrid ACO-PSO')
    ax[2].set_title('Hybrid ACO-PSO Results')
    ax[2].set_xlabel('Best Distance')
    ax[2].set_ylabel('Time Window Violations')

    plt.tight_layout()
    plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/c101.txt'

In [ ]:
plot_best_route(locations, best_route_aco)
plot_best_route(locations, best_route_pso)
plot_best_route(locations, best_route_aco_hybrid)

In [ ]:
import plotly.graph_objects as go

def plot_route(depot, customers, route):
    # Prepare data for plotting
    x = [depot[0]]  # Start with depot
    y = [depot[1]]

    # Append customer coordinates based on route
    for customer_id in route:
        coords = customers[customer_id]['coordinates']
        x.append(coords[0])
        y.append(coords[1])

    # Close the loop by returning to the depot
    x.append(depot[0])
    y.append(depot[1])

    # Create 3D plot
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=x,
        y=y,
        z=list(range(len(route) + 1)),  # Z-axis as visit order
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=2)
    ))

    # Update layout
    fig.update_layout(
        title="3D Route Visualization",
        scene=dict(
            xaxis_title="X Coordinate",
            yaxis_title="Y Coordinate",
            zaxis_title="Visit Order"
        ),
        margin=dict(l=0, r=0, b=0, t=40)
    )

    fig.show()
